In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.tools import tool
from langchain.schema.runnable import RunnablePassthrough
from pydantic import BaseModel, Field
from typing import List, Optional
import requests
from langchain.schema.agent import AgentFinish

In [5]:
class productDetails(BaseModel):
    product_category: str = Field(description="The name of the product required")
    product_id : int = Field(description="The id of the desired product to look for ")
class DiscountDetails(BaseModel):
    min_discount: float = Field(description="The minmum discount price of the product to look for")
    max_discount : float = Field(description="The maxmum discount price of the product to look for ")

In [6]:
@tool(args_schema=productDetails)
def get_product_information(product_category:str, product_id:int) ->dict:
     """get details of a given product name of the given url"""
     url = "http://localhost/ecommerce_data/data.json"
     response = requests.get(url)
     if response.status_code ==200:
          data= response.json()
          for product in data[product_category]:
               if product['id'] == product_id:
                    return product
               return {"error": "Product not found"}
          return {"error": f"Sorry, I couldn't find a product with ID {product_id} in the {product_category} category. Can you double-check or try again?"}
     else:
          raise Exception(f"API Request failed with status code: {response.status_code}")

In [7]:
@tool(args_schema=DiscountDetails)
def get_discounted_products(min_discount:float, max_discount:float) -> dict:
    """Search for all products that have discounts available"""
    url = "http://localhost/ecommerce_data/data.json"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        discounted_products = []
        for product in data['discountItems']:
            if min_discount <= float(product['price']) <= max_discount:
                discounted_products.append({
                    
                    'name': product['name'],
                    'price': product['price'],
                    'cover': product['cover']
                })
            if not discounted_products:
                return {"error": "Sorry, no discounted products were found in that price range."}
        return discounted_products
        
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")


In [95]:
def classfy_intent(user_input)-> str:
    """Classifies the user's intent based on the input."""
    if "min_discount" in user_input or "max_discount" in user_input:
        return "get_discounted_products"
    elif "product_id" in user_input or "details" in user_input:
        return "get_product_information"
    else:
        return "fallback"

In [65]:

def route_to(result):
    if isinstance(result, AgentFinish):
        return result.return_values['output']
    else:
        intent = classfy_intent(result.tool_input)
        tools = {
            "get_discounted_products": get_discounted_products, 
            "get_product_information": get_product_information,
        }
        if intent in tools:
            return tools[intent].run(result.tool_input)
        else:
            return "I'm sorry, I didn't quite understand that. Can you please clarify?"

In [66]:
from langchain.tools.render import format_tool_to_openai_function

functions = [
   format_tool_to_openai_function(get_product_information), 
   format_tool_to_openai_function(get_discounted_products)
]

In [67]:
model = ChatOpenAI(temperature=0).bind(functions=functions)
prompt =ChatPromptTemplate.from_messages([
    ("system", "You are Tata, a friendly and Expert business assistant who helps Rhodrick with selling a wide variety of products.  You know everything about product prices, discounts, and the best offers available. Your goal is to provide personalized recommendations and assist customers efficiently and cheerfully."),
    ("system", "When you don't know something, politely say 'I'm not sure about that, but I can try to find more information for you!' instead of making up suggestions that are not valid."),
    ("system", "If a customer asks a follow-up question or requests more information, do your best to provide a helpful and accurate response."),
    ("user", "{input}"),
])

In [111]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser() | route_to

In [112]:
response = chain.invoke({"input":"show me productItems that have id 1"})


In [113]:
response

{'id': 1,
 'discount': 50,
 'cover': 'http://169.254.228.82/ecommerce_data/images/flash/flash-1.png',
 'name': 'Shoes',
 'price': 100}